# Bilingual dictionary enrichment via graph completion

Current

In [1]:
import logging
import sys

logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import json
import numpy as np
import re

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import networkx as nx
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import requests

## Language codes

In [ ]:
from numpy import na
import pandas as pd
lang_codes = pd.read_csv('./files/language-codes-full_csv.csv', na_values = 0)
lang_codes = lang_codes[['alpha3-b','alpha2']]
lang_codes = lang_codes.dropna()


lang_codes = [{i[0]:i[1] for i in np.array(lang_codes)}, {i[1]:i[0] for i in np.array(lang_codes)}]

with open ('./files/lang_codes.json', 'w') as f:
    json.dump(lang_codes, f)

In [100]:
with open ('./files/lang_codes.json', 'r') as f:
    lang_codes = json.load(f)

def l(lang, mode=3):
    mode = mode % 2
    if len(lang)==2:
        if lang in lang_codes[mode]:
            return lang_codes[mode][lang]
        else:
            return lang
    else:
        return lang
l('tt', 3)

'tat'

## Loading dictionaries

### Git - not relevant

In [2]:
import git

def clone_folder(name='apertium-trunk', target='./data/'):
    repo = git.Repo.clone_from('https://github.com/apertium/'+name+'/', target)
    for i in git.objects.submodule.root.RootModule(repo).list_items(repo):
        git.Repo.clone_from('https://github.com/apertium/'+i.name, './data/'+i.name)
        logging.info(i.name)

clone_folder()

Took 3 hours to clone aprtium-trunk

### PyGithub

** Load user with login and password from secret file **

In [101]:
from github import Github

with open ('secure.json') as f:
    SECRET = json.loads(f.read())

github = Github(SECRET['USER'], SECRET['PASSWORD'])

user = github.get_user('apertium')

In [ ]:
user.get_repos()

** Generator ** : yield all repos that match name pattern

In [102]:
def repo_names(user):
    for repo in user.get_repos():
        if re.match('apertium-[a-z]{2,3}(_[a-zA-Z]{2,3})?-[a-z]{2,3}(_[a-zA-Z]{2,3})?', repo.name):
            yield repo.name

Looks like heavy function. But I don't see any improvements yet, except for having certain repo for all bidix copies. But this one above is the most up-to-date. It filters not languages pair repos, it is needed not to look for bidix where it can't be. Function saves a lot of time.

In [18]:
%time w = list(repo_names(user))

Wall time: 26.8 s


** Find bidix **

In [103]:
def bidix_url(repo):
    bidix = repo.name+'.'+repo.name.replace('apertium-','')+'.dix'
    for i in repo.get_dir_contents('/'):
        if re.match('apertium-.*?\.[a-z]{2,3}(_[a-zA-Z]{2,3})?-[a-z]{2,3}(_[a-zA-Z]{2,3})?.dix', i.path):
            return i.download_url

In [19]:
%time bidix_url(github.get_repo(user.name+'/'+w[22]))

Wall time: 1.31 s


'https://raw.githubusercontent.com/apertium/apertium-cat-srd/master/apertium-cat-srd.cat-srd.dix'

In [104]:
def bidix_url(repo):
    for i in sorted(repo.get_dir_contents('/'), key = lambda x: (len(x.path), 1000-ord(('   '+x.path)[-3])), reverse=True):
        if re.match('apertium-.*?\.[a-z]{2,3}(_[a-zA-Z]{2,3})?-[a-z]{2,3}(_[a-zA-Z]{2,3})?.dix$', i.path):
            return i.download_url
        elif len(i.path) < 23:
            return None

In [27]:
%time bidix_url(github.get_repo(user.name+'/'+w[22]))

Wall time: 709 ms


'https://raw.githubusercontent.com/apertium/apertium-cat-srd/master/apertium-cat-srd.cat-srd.dix'

Speed is ok. 

Check repos for bidixes

In [9]:
def download(user):
    for repo_name in repo_names(user):
        bidix = bidix_url(github.get_repo(user.name+'/'+repo_name))
        if not bidix:
            print (repo_name)

download(user)

apertium-en-lv
apertium-est-nor
apertium-ita-srd
apertium-ky-en
apertium-lex-tools
apertium-on-github
apertium-ru-cu
apertium-sc-pt
apertium-urd-pan


All these repos do not have bidixes except for the last one. There are two of them. So that's strange.

** XML reading ** : return xml tree object. Read file with request from github and return object

In [105]:
def tree(url):
    response = requests.get(url)
    return ET.fromstring(response.content)

We need to check bidixes, because errors occur.

** Errors **

- eo-bg : strange header section with sdefs    ParseError: mismatched tag: line 10, column 4
- eo-fa : same, looks like < sdef n="n" > should be < sdef n="n"/>
- eo-pl : same
- fin-fra : < !-- \n {{{ Punctuatkion and stuff \n {{{ puncts --> ParseError: not well-formed (invalid token): line 152, column 4
- pl-lv : Possibly, not closed < alphabet > (dictionary) ParseError: mismatched tag: line 296, column 2
- sah-eng : " [< Russ. " in text, this is parsed as tag : ParseError: not well-formed (invalid token): line 339, column 114

In [ ]:
tree('https://raw.githubusercontent.com/apertium/apertium-sah-eng/master/apertium-sah-eng.sah-eng.dix')

Loading all files (even without parsing) takes a lot of time (12 minutes on Windows). So to reduce time in case of gathering files instead of one folder on github we need to reduce number of languages we download.

** Only relevant for certain language pair **

There are **164 ** pairs at this moment

In [38]:
def get_repos_for_pair(user, l1, l2, n=[2]):
    logging.info('Start')
    G = nx.DiGraph()
    lg = (l(l1,3), l(l2,3))
    pair_list = []
    for name in repo_names(user):
        pair_list.append(name)
        w = re.findall('.*?-([a-zA-Z_]{2,7})-([a-zA-Z_]{2,7})$', name)[0]
        w = (l(w[0],3), l(w[1],3))
        if w[0] == lg[0] or w[1] == lg[1]: G.add_edge(w[0],w[1])
        elif w[0] == lg[1] or w[1] == lg[0]: G.add_edge(w[1],w[0])
        else:
            G.add_edge(w[0],w[1])
            G.add_edge(w[1],w[0])
    if (lg[0], lg[1]) in G.edges(): G.remove_edge(lg[0], lg[1])
    if (lg[1], lg[0]) in G.edges(): G.remove_edge(lg[1], lg[0])
    logging.info('Graph')
    
    for i in n:
        w = nx.single_source_shortest_path_length(G, lg[0], cutoff=i)
        v = nx.single_source_shortest_path_length(G, lg[1], cutoff=i)
        nodes = list((set(w.keys())&set(w.keys()) )| set([lg[0],lg[1]]))
        H = G.subgraph(nodes)
        logging.info('Length: {}\tNodes: {}'.format(i, len(nodes)))
    return G

** Final loading **

In [106]:
def get_repos_for_pair(user, l1, l2, n=[2]):
    logging.info('Start')
    pair_list = []
    G = nx.Graph()
    lg = (l(l1,3), l(l2,3))
    for name in repo_names(user):
        pair_list.append(name)
        w = [l(i,3) for i in re.findall('.*?-([a-zA-Z_]{2,7})-([a-zA-Z_]{2,7})$', name)[0]]
        G.add_edge(w[0],w[1])
    logging.info('Built graph')
    
    for i in n:
        w = nx.single_source_shortest_path_length(G, lg[0], cutoff=i)
        v = nx.single_source_shortest_path_length(G, lg[1], cutoff=i)
        H = G.subgraph(w.keys())
        H.remove_node(lg[0])
        H2 = G.subgraph(v.keys())
        H2.remove_node(lg[1])
        nodes = set(nx.node_connected_component(H, lg[1])) & set(nx.node_connected_component(H2, lg[0])) | set([lg[0], lg[1]])
        logging.info('Length: {}\tNodes: {}'.format(i, len(nodes)))
        
    i = int(input('Choose max halfway: '))
    w = nx.single_source_shortest_path_length(G, lg[0], cutoff=i)
    v = nx.single_source_shortest_path_length(G, lg[1], cutoff=i)
    H = G.subgraph(w.keys())
    H.remove_node(lg[0])
    H2 = G.subgraph(v.keys())
    H2.remove_node(lg[1])
    nodes = set(nx.node_connected_component(H, lg[1])) & set(nx.node_connected_component(H2, lg[0]))| set([lg[0], lg[1]])
    return set(nodes), pair_list

In [108]:
def filter_names(nodes:set, pair_list):
    for repo in pair_list:
        langs = re.findall('apertium-(.*?)-(.*?)$', repo)[0]
        if l(langs[0]) in nodes and l(langs[1]) in nodes:
            yield repo

In [107]:
def download_language_pair_support(user, l1, l2, n=[2]):
    nodes, pair_list = get_repos_for_pair(user, l1, l2, n)
    logging.info('Started loading')
    for repo_name in filter_names(nodes, pair_list):
        logging.info(repo_name)
        url = bidix_url(github.get_repo(user.name+'/'+repo_name))
        if url:
            lang = re.findall('.*?\.([a-zA-Z_]{2,7})-([a-zA-Z_]{2,7})\.dix$', url)
            if lang:
                l1, l2 = lang[0][0], lang[0][1]
                try:
                    t = tree(url)
                    yield t
                except:
                    #print(l1, l2)
                    pass
            else:
                print (url)
    logging.info('Finished')

In [46]:
list(download_language_pair_support(user, 'urd','hin', n=[2,3,4]))

2018-05-17 17:39:43,298 | INFO : Start
2018-05-17 17:40:09,223 | INFO : Built graph
2018-05-17 17:40:09,223 | INFO : Length: 2	Nodes: 4
2018-05-17 17:40:09,231 | INFO : Length: 3	Nodes: 4
2018-05-17 17:40:09,234 | INFO : Length: 4	Nodes: 4
Choose max halfway: 4
2018-05-17 17:40:18,115 | INFO : Started loading
2018-05-17 17:40:23,386 | INFO : Finished


[<Element 'dictionary' at 0x0000017B022DD1D8>,
 <Element 'dictionary' at 0x0000017B7EC54548>,
 <Element 'dictionary' at 0x0000017B009780E8>,
 <Element 'dictionary' at 0x0000017B00E77188>]

In [47]:
list(download_language_pair_support(user, 'hin','urd', n=[2,3,4]))

2018-05-17 17:40:27,545 | INFO : Start
2018-05-17 17:41:00,468 | INFO : Built graph
2018-05-17 17:41:00,468 | INFO : Length: 2	Nodes: 4
2018-05-17 17:41:00,468 | INFO : Length: 3	Nodes: 4
2018-05-17 17:41:00,479 | INFO : Length: 4	Nodes: 4
Choose max halfway: 4
2018-05-17 17:41:03,828 | INFO : Started loading
2018-05-17 17:41:08,687 | INFO : Finished


[<Element 'dictionary' at 0x0000017B0083ADB8>,
 <Element 'dictionary' at 0x0000017B047EAEF8>,
 <Element 'dictionary' at 0x0000017B03527D68>,
 <Element 'dictionary' at 0x0000017B05573F48>]

In [109]:
def save_all(user):
    logging.info('Start')
    for repo_name in repo_names(user):
        url = bidix_url(github.get_repo(user.name+'/'+repo_name))
        if url:
            lang = re.findall('.*?\.([a-zA-Z_]{2,7})-([a-zA-Z_]{2,7})\.dix$', url)
            if lang:
                l1, l2 = lang[0][0], lang[0][1]
                try:
                    t = tree(url)
                    yield t
                except:
                    pass
    logging.info('Finish')

In [ ]:
list(save_all(user))[0]

2018-05-17 18:52:17,411 | INFO : Start


In [30]:
tree('https://raw.githubusercontent.com/apertium/apertium-urd-hin/master/apertium-urd-hin.urd-hin.dix')

<Element 'dictionary' at 0x000001CCDB4114A8>

## Object classes

** Word **

- lemma : lemma
- lang : language
- pos : part of speech

In [53]:
class Word:
    def __init__(self, lemma, lang, s=[]):
        self.lemma = lemma
        self.lang = lang
        self.s = set(s)
        self.slist = s
    
    def __str__(self):
        return str(self.lang)+'_'+str(self.lemma)+'_'+str('-'.join(self.slist))
    
    __repr__ = __str__
    
    def __eq__(self, other):
        return self.lemma == other.lemma and self.lang == other.lang and self.s == other.s
    
    def __hash__(self):
        return hash(str(self))

## Parsing

### Bidix parsing

In [87]:
%time T = tree('https://raw.githubusercontent.com/apertium/apertium-bel-rus/master/apertium-bel-rus.bel-rus.dix')

Wall time: 2.26 s


In [88]:
def one_word(word, lang):
    s = word.findall('.//s')
    s = [i.attrib['n'] for i in s]
    return Word(word.text, lang, s)

In [98]:
def parse_bidix (tree, l1, l2):
    tree = tree.find('section')
    for e in tree:
        if 'n' in e.attrib:
            side = e.attrib['n']
        else:
            side = None
        p = e.find('p')
        yield one_word(p.find('l'), l1), one_word(p.find('r'), l2), side

In [99]:
% time len(list(parse_bidix (T, 'bel','rus')))

Wall time: 757 ms


48880